In [1]:
import pandas as pd

In [2]:
%load_ext rpy2.ipython

# Question Persistence Analysis

## Select course and load data set

In [3]:
data_dir = '/Users/benny/data/L@S_2021'
course = 'microbiology'

In [4]:
data_set_filename = f'persistence_{course}.txt'
data_set = pd.read_csv( f'{data_dir}/{data_set_filename}', sep='\t' )

In [5]:
data_set

,student,question,course_page_number,unit_page_number,module_page_number,page_question_number,question_type,answered,correct,persisted
0,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135e77,18,16,5,1,AG_FITB,1,0,1
1,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135f91,18,16,5,12,AG_Matching,1,0,1
2,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135fa1,18,16,5,13,AG_Matching,1,0,1
3,5f48174400b3a42fe2c6e6a5,5f2e4e5de3a4d9432b135fd3,18,16,5,15,AG_FITB,1,0,1
4,5f48174400b3a42fe2c6e6a5,5f2e4e2ee3a4d9432b12f574,15,13,2,8,HA_MC,1,0,1
...,...,...,...,...,...,...,...,...,...,...
10515,600e1302e16a3a3764c6730a,5f2e4e89e3a4d9432b13bf88,28,26,6,6,AG_FITB,1,0,1
10516,600e1302e16a3a3764c6730a,5f2e4e8ae3a4d9432b13bfae,28,26,6,7,AG_Matching,1,0,1
10517,600e1302e16a3a3764c6730a,5f2e4e8ae3a4d9432b13bfce,28,26,6,9,AG_Matching,1,0,1
10518,600e1302e16a3a3764c6730a,5f2e4e8ae3a4d9432b13bff0,28,26,6,10,AG_FITB,1,0,1


## Mean persistence

In [6]:
data_set.groupby( 'question_type' ).mean().sort_values( by='persisted', ascending=False )

,course_page_number,unit_page_number,module_page_number,page_question_number,answered,correct,persisted
question_type,,,,,,,
AG_Matching,52.348024,50.348024,4.025076,5.040274,1.0,0.0,0.993161
HA_MC,42.458378,40.458378,4.044456,8.753631,1.0,0.0,0.988665
HA_MCMS,44.894057,42.894057,4.555556,10.036176,1.0,0.0,0.984496
HA_Pulldown,52.822540,50.822540,5.043734,17.245585,1.0,0.0,0.978974
AG_FITB,43.203830,41.203830,4.451513,4.966028,1.0,0.0,0.900556
HA_DnD_Image,54.000000,52.000000,6.000000,32.877193,1.0,0.0,0.830409
HA_DnD_Table,39.890625,37.890625,7.218750,14.895833,1.0,0.0,0.791667


## Regression model

In [7]:
%%R
library( lme4 )

R[write to console]: Loading required package: Matrix



Standardize the continuous variables.

In [8]:
for col in [ 'course_page_number', 'unit_page_number', 'module_page_number', 'page_question_number' ]:
    data_set[ col ] = ( data_set[ col ] - data_set[ col ].mean() ) / data_set[ col ].std()

In [9]:
data_set.to_csv( '/tmp/to_r.csv', index=False )

In [10]:
%%R
df <- read.csv( '/tmp/to_r.csv' )

In [11]:
%%R
lme.model <- glmer( persisted ~ course_page_number + unit_page_number + module_page_number + page_question_number + question_type
                              + (1|student) + (1|question), family=binomial(link=logit),
                    data=df,
                    control=glmerControl( optimizer="bobyqa", optCtrl=list(maxfun=2e4) ) )
summary( lme.model )

R[write to console]: fixed-effect model matrix is rank deficient so dropping 1 column / coefficient



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
persisted ~ course_page_number + unit_page_number + module_page_number +  
    page_question_number + question_type + (1 | student) + (1 |      question)
   Data: df
Control: glmerControl(optimizer = "bobyqa", optCtrl = list(maxfun = 20000))

     AIC      BIC   logLik deviance df.resid 
  2113.0   2200.1  -1044.5   2089.0    10508 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-18.0105   0.0339   0.0646   0.1151   2.3184 

Random effects:
 Groups   Name        Variance Std.Dev.
 question (Intercept) 0.7445   0.8629  
 student  (Intercept) 3.1275   1.7685  
Number of obs: 10520, groups:  question, 942; student, 96

Fixed effects:
                          Estimate Std. Error z value Pr(>|z|)    
(Intercept)                3.09502    0.27184  11.386  < 2e-16 ***
course_page_number         0.17915    0.09617   1.863   0.0625 .  
module_p